# Feature Analysis - Apply Selection on Calculated Features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder #string labels to int
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA as PCA
from sklearn.feature_selection import chi2, mutual_info_classif, SelectKBest

In [7]:
import platform
platform.python_version()


'2.7.15'

## Load Data

In [2]:
sampling_rate = 128
path = '../data/preprocessed/'
features = pd.read_csv(path+'featuresCarlo.csv', ',')
features.head()

,AF3_beta,F7_beta,F3_beta,FC5_beta,T7_beta,P7_beta,O1_beta,O2_beta,P8_beta,T8_beta,...,FC5_mu,T7_mu,P7_mu,O1_mu,O2_mu,P8_mu,T8_mu,FC6_mu,F4_mu,F8_mu
0,9.617036,9.626228,9.590239,9.623327,9.589991,9.590804,9.567983,9.582804,9.574996,9.617209,...,9.972018,9.978088,9.946640,9.907507,9.920496,9.902075,9.984438,9.982150,9.949355,9.957458
1,10.281531,10.350626,10.272987,10.294057,10.321789,10.265471,10.242162,10.268789,10.245133,10.339490,...,10.157041,10.185575,10.147617,10.127310,10.135965,10.117657,10.178475,10.168443,10.142686,10.148809
2,9.704346,9.717879,9.685493,9.684669,9.691322,9.686065,9.678658,9.688431,9.678609,9.711365,...,9.624257,9.659974,9.642986,9.636403,9.645297,9.629779,9.650872,9.650194,9.641817,9.641468
3,10.062664,10.014825,9.952124,9.974973,9.990027,9.940493,9.940601,9.947620,9.951701,10.029456,...,10.596914,10.614033,10.563460,10.550088,10.571490,10.559683,10.656713,10.620954,10.602026,10.608514
4,9.925013,9.979357,9.939241,9.950245,9.985936,9.932586,9.921234,9.930336,9.923825,9.978828,...,10.294519,10.312448,10.267675,10.250802,10.261787,10.250422,10.305293,10.302618,10.269625,10.282871


In [3]:
features.describe()

,AF3_beta,F7_beta,F3_beta,FC5_beta,T7_beta,P7_beta,O1_beta,O2_beta,P8_beta,T8_beta,...,FC5_mu,T7_mu,P7_mu,O1_mu,O2_mu,P8_mu,T8_mu,FC6_mu,F4_mu,F8_mu
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,...,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,9.503379,9.501398,9.497009,9.496133,9.498933,9.496482,9.495756,9.498005,9.496860,9.501696,...,9.681783,9.683564,9.681701,9.679060,9.683999,9.682901,9.689437,9.688723,9.682604,9.686821
std,0.240572,0.246017,0.236666,0.233880,0.244878,0.233998,0.232484,0.235911,0.233964,0.245249,...,0.255031,0.268336,0.254650,0.253625,0.258507,0.254622,0.269650,0.266790,0.262236,0.263300
min,9.043077,8.896678,8.911320,8.955319,8.905453,8.929723,8.915961,8.917642,8.914699,8.881114,...,9.200563,9.114560,9.139225,9.152171,9.145037,9.132165,9.056587,9.105761,9.103077,9.138060
25%,9.422197,9.424019,9.424810,9.424938,9.425865,9.425995,9.427918,9.425859,9.423194,9.425590,...,9.591891,9.591689,9.591146,9.590364,9.587952,9.587526,9.589611,9.589586,9.583695,9.589597
50%,9.433496,9.432888,9.432687,9.431912,9.432380,9.434322,9.433273,9.433857,9.434847,9.434682,...,9.603508,9.600854,9.604653,9.599981,9.603008,9.604577,9.604735,9.604796,9.598517,9.605205
75%,9.466994,9.467997,9.463925,9.470371,9.459698,9.459802,9.461531,9.478604,9.464113,9.472344,...,9.640113,9.646433,9.641194,9.637056,9.650339,9.650284,9.655533,9.653226,9.657403,9.648331
max,11.075647,11.091786,11.064608,11.057513,11.086851,11.047443,11.040735,11.058450,11.046835,11.091721,...,11.108204,11.166813,11.093494,11.095956,11.108408,11.084503,11.184678,11.164897,11.122122,11.138526


## Apply LDA  

In [4]:
# Prepare data
X = features[features.columns.difference(['id', 'label'])]
y = features.label

In [5]:
lda = LDA()
lda.fit(X, y)
scores_lda = lda.scalings_

In [6]:
scores_lda

array([[-25.39014435, -43.82011675],
       [ 31.93901529,  45.79650856],
       [ 43.31175936, -81.99427624],
       [-50.0095103 ,  92.44990251],
       [ 40.84970991,  73.20594772],
       [ 20.46626457, -72.51143915],
       [-91.53200403,  34.22547228],
       [ 28.9413079 , -55.27231667],
       [-28.1183065 ,  15.79666009],
       [  1.20012942, -33.79497838],
       [ 20.22748459,  13.48291609],
       [-57.29284707, -10.90682576],
       [-23.60166519,  -5.89877478],
       [ 50.39196599,  -4.77834132],
       [ -8.61918319,  41.39359998],
       [  1.6092839 , -82.00224567],
       [ 35.77495462, -19.06305549],
       [ -0.97735896,  74.02300949],
       [ -7.84172705, -42.99865741],
       [ 27.19412626,  48.32567872],
       [-15.88112101,  90.12235204],
       [  7.58338287, -65.57467156],
       [  1.23111606,  -2.5615286 ],
       [ -2.05649772,  18.26871642],
       [ 62.04472992, -66.95004773],
       [-58.91280677,  43.40072451]])